In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
import os

In [6]:
data_file = os.path.join(os.curdir, 'data', 'sample_linear_regression_data.txt')

In [7]:
training = spark.read.format('libsvm').load(data_file)

In [8]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [9]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol='prediction')

In [11]:
lr_model = lr.fit(training)

In [12]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lr_model.intercept

0.14228558260358093

In [14]:
training_summary = lr_model.summary

In [15]:
training_summary.r2

0.027839179518600154

In [16]:
training_summary.rootMeanSquaredError

10.16309157133015

In [19]:
# the example above took everything as training data which is wrong,
# so lets try with the split

total_data = spark.read.format('libsvm').load(data_file)
training_data, testing_data = total_data.randomSplit([0.7, 0.3])

In [21]:
training_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|
|-15.732088272239245|(10,[0,1,2,3,4,5,...|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|
|-15.359544879832677|(10,[0,1,2,3,4,5,...|
|-15.348871155379253|(10,[0,1,2,3,4,5,...|
+----------

In [22]:
training_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                365|
|   mean| 0.8696659611190214|
| stddev|  9.904760390417673|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [23]:
testing_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                136|
|   mean|-1.3876968815349486|
| stddev|  11.22646280947795|
|    min|-28.046018037776633|
|    max|  22.31738046492344|
+-------+-------------------+



In [24]:
correct_model = lr.fit(training_data)

In [25]:
test_results = correct_model.evaluate(testing_data)

In [27]:
test_results.rootMeanSquaredError

11.573966748374271

In [29]:
unlabeled_data = testing_data.select('features')

In [30]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [31]:
predictions = correct_model.transform(unlabeled_data)

In [32]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-1.1374471760288714|
|(10,[0,1,2,3,4,5,...| 2.4609797365335355|
|(10,[0,1,2,3,4,5,...| -2.724765294000195|
|(10,[0,1,2,3,4,5,...| 0.2082570032940011|
|(10,[0,1,2,3,4,5,...|  4.562972853712567|
|(10,[0,1,2,3,4,5,...| 1.5169007216632817|
|(10,[0,1,2,3,4,5,...| 0.5359532994170475|
|(10,[0,1,2,3,4,5,...|-0.7883179477752581|
|(10,[0,1,2,3,4,5,...|  2.294214229063558|
|(10,[0,1,2,3,4,5,...| 1.1836567288297446|
|(10,[0,1,2,3,4,5,...|  2.655034643445085|
|(10,[0,1,2,3,4,5,...| 2.9286990327860942|
|(10,[0,1,2,3,4,5,...|-1.1786762723841209|
|(10,[0,1,2,3,4,5,...|  0.961149206065264|
|(10,[0,1,2,3,4,5,...|  4.161214730669445|
|(10,[0,1,2,3,4,5,...|-0.3997953144637688|
|(10,[0,1,2,3,4,5,...| -2.277281629056755|
|(10,[0,1,2,3,4,5,...| 2.2980900467313194|
|(10,[0,1,2,3,4,5,...| 0.0861207175739046|
|(10,[0,1,2,3,4,5,...| 1.1061393959746408|
+----------

In [33]:
ecom_data_file = os.path.join(os.curdir, 'data', 'Ecommerce_Customers.csv')

In [34]:
ecom_data = spark.read.csv(ecom_data_file)

In [35]:
ecom_data.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|              _c0|                 _c1|        _c2|               _c3|               _c4|               _c5|                 _c6|                _c7|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              501|                 501|        501|               501|               501|               501|                 501|                501|
|   mean|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|            Email|0001 Mack MillNor..